# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
# alt.data_transformers.enable('data_server')
alt.data_transformers.disable_max_rows()
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [ ]:
start_date = pd.to_datetime("2020-04-13")

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.head()

## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos la fecha por cada grupo etáreo.

In [ ]:
alt.Chart(fallecidos_etareo,title="fallecidos según fecha agrupados por edad").mark_line().encode(
    x="fecha:T",
    y=alt.Y("fallecidos"),
    color="grupo_de_edad:N"
)

**Comentarios:**  Notamos  que hay grandes ingresos de fallecidos al sistema a mitad de Julio, antes de ese momento la diferencia entre las curvas cambia, después la diferencia no cambia tan rápido. El grupo con la mayor cantidad de casos es de 70-79, luego 80-89 y 60-69 en el último tiempo.

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [ ]:

pobla=activos.loc[:,["codigo_comuna","poblacion"]].drop_duplicates()

alt.Chart(pobla,title="Histograma de la población de las comunas de Chile").mark_bar().encode(
   alt.X("poblacion:Q" , bin=alt.Bin(extent=[0, 700000], step=3000), title= "población(agrupada en intervalos)"),
   y=alt.Y('count()',title="cantidad de comunas"),

).properties(
    width=1250,
    height=500
)


**Comentarios:** Se hizo un histograma de la notación. Notamos como la mayor parte de las comunas tienen una concetración entre 0 y 36000 app personas, si bien hay comuna 

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [ ]:

alt.Chart(fallecidos_etareo,title="fallecidos según grupo de edad").mark_area().encode(
    x="fecha:T",
    y=alt.Y("fallecidos"),
    color=alt.Color("grupo_de_edad:N",title="grupo de edad")
)

**Comentarios:** Notamos momentos en que hay grandes ingresos de fallecidos al sistema(por ej. a mitad de Julio), también que la mayor proporción de muertes respecto al total se ecunetra en las personas mayores de 60 años, especialemente, entre los grupos de 60-69, 70-79 y 80-89 años, con csntidades similares entre estos.

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [ ]:

alt.Chart(confirmados, title="número de casos confirmados a nivel nacional según fecha").mark_line().encode(
    x="fecha:T",
    y=alt.Y("sum(casos_confirmados)",title="número de casos confirmados")
)

In [ ]:
alt.Chart(activos,title="casos activos a nivel nacional según fecha ").mark_line().encode(
    x="fecha:T",
    y=alt.Y("sum(casos_activos)",title="número de casos activos")
)

In [ ]:
alt.Chart(fallecidos,title="número de fallecidos a nivel nacional según fecha").mark_line().encode(
    x="fecha:T",
    y=alt.Y("sum(fallecidos)",title="cantidad de fallecidos")
)

**Comentarios:** Notamos que los casos confirmados, y el número de fallecidos tienen un comportamiento similar que no presentan los casos activos, esto esta relacionados los casos confirmados y fallecidos son acumulativos, mientras que los casos activos no. Notamos en los casos activos, tenemos un peak a mediados de Junio y luego disminuye, pero no de manera constante, hasta que en Noviembre practicamente se mantiene contante.

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [ ]:

tasa=confirmados.groupby(["region","fecha"]).apply(lambda df: (df["tasa"]*df["poblacion"]).sum()/df["poblacion"].sum() ).to_frame().rename(columns={0:"tasa"}).reset_index()
tasa["fecha"]= [str(d.date()) for d in tasa["fecha"]]

alt.Chart(tasa,title=" tasa de incidencia según región a lo largo del tiempo").mark_rect().encode(
    x=alt.X("region", title="Región"),
    y=alt.Y("fecha:O",title="fecha en formato Año-Mes-Día"),
    color=alt.Color("tasa:Q")
)

**Comentarios:** Notamos como cada Región han habido comportamientos distintos, con algunas con contagios más tempranos y otros más tardios,así como también con variedad en los valores de tasas  , destacan los números de la región de Magallanes, donde si bien el aumento comenzó más tarde que en alguna de las otras regiones, es el que tiene la tasa más alta. 

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [ ]:
#cambiar color, arreglar fechas, relación es simplemente ratio población, 
#tamño pelotitas
comienzo = pd.to_datetime("2020-04-13")
fin=  pd.to_datetime("2020-11-06")
datos_comienzo=confirmados.loc[(confirmados["fecha"]==comienzo)]
datos_fin=confirmados.loc[confirmados["fecha"]==fin]
alt.Chart(datos_comienzo,title="tasa vs casos confirmados para el 13 de Abril de 2020").mark_circle().encode(
    x=alt.X("casos_confirmados",title="casos confirmados"),
    y=alt.Y("tasa", title="tasa (casos confirmados por 100.000 hab)"),
    color=alt.Color("region" ,scale=alt.Scale(scheme="category20")),
    size="poblacion",
    tooltip=["comuna"]
    
)

In [ ]:
alt.Chart(datos_fin,title="tasa vs casos confirmados  para el el 6 de Noviembre de 2020").mark_circle().encode(
    x=alt.X("casos_confirmados",title="casos confirmados"),
    y=alt.Y("tasa", title="tasa (casos confirmados por 100.000 hab)"),
    color=alt.Color("region" ,scale=alt.Scale(scheme="category20")),
    size="poblacion",
    tooltip=["comuna","poblacion"]
).interactive()

**Comentarios:**   
Se pone el tamaño proporcional ya que nos permite notar como se relaciona los casos confirmados con la población de la comuna, con en general,  mayor cantidad de casos confirmados para comunas más grandes(se nota claramente).

En ambos se logra reconocer regiones donde predominan las comunas de la región metropolitana, especialmente en la segunda fecha, correspondiendo en este caso a comunas con tasa más alta, y con poblaciones no ínfimas.

para la primera fecha se observa un outlier que corresponde a la comuna de Cabo de Hornos.

para la segunda fecha se observan valores de tasa/casos confirmados mayores. Un outlier en puente alto de casos confirmados.





## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [ ]:

alt.Chart(activos,title= "casos activos por región según fecha" ).mark_line().encode(
    x='fecha',
    y=alt.Y('sum(casos_activos)',title="casos activos"),
    color=alt.Color('region',scale=alt.Scale(scheme="category20c"))
)

In [ ]:
base = alt.Chart(activos).mark_line().encode(
    x="fecha:T",
    y=alt.Y("sum(casos_activos)",title="casos activos")
).properties(
    width=280,
    height=300
).facet("region",columns=4,title="Casos activos en cada región según fecha").resolve_scale(x="independent",y='independent')

display(base)



**Comentarios:** 
1. * Pros: Permite tener una idea de como se compara el comportamiento entre  regiones como "grupo". 
   * Contras: No se aprecia claramente la curva para alguna regiones
   
2. * Pros: Se observa claramente cual es el el comportamiento por región. 
   * Contras: Es más difícil comparar.


## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [ ]:
region_names = {
    "Araucanía": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
}

In [ ]:

mean_activos=activos[["codigo_region","casos_activos"]].groupby(["codigo_region"]).agg("mean").reset_index()

max_fallecidos=fallecidos.replace(region_names).groupby(["region","fecha"]).agg(sum).reset_index().groupby(["region"]).agg(max).reset_index()

max_confirmados=confirmados[["region","region_id","fecha","casos_confirmados"]].groupby(["region","region_id","fecha"]).agg(sum).reset_index().rename(columns={"region_id":"codigo_region"}).groupby(["codigo_region"]).agg(max).reset_index()

merged=mean_activos.merge(max_confirmados,on=["codigo_region"]).merge(max_fallecidos,on=["region"])


In [ ]:
base_fallecidos=alt.Chart(original,title="máximo de fallecidos").mark_bar(color="blue").encode(
    y=alt.Y("region",axis=alt.Axis(labels=False), title=None),
    x=alt.X('fallecidos:Q', title="máximo de fallecidos"),
    
).properties(width=400,height=700 )

texto_fallecidos=base_fallecidos.mark_text( align='left',
    baseline='middle',
    dx=3 ).encode(
    text=alt.Text('fallecidos'),
)
fallecidosgraph=base_fallecidos + texto_fallecidos

base_confirmados=alt.Chart(original,title="máximo de casos confirmados").mark_bar(color="red").encode(
    y=alt.Y("region"),
    x=alt.X('casos_confirmados:Q',title="máximo de casos confirmados"),
    text=alt.Text('casos:Q'),
).properties(width=400 ,height=700)

texto_confirmados=base_confirmados.mark_text( align='left',
    baseline='middle',
    dx=3 ).encode(
    text=alt.Text('casos_confirmados'),
)
confirmadosgraph=base_confirmados + texto_confirmados

base_activos=alt.Chart(original,title="promedio de casos activos").mark_bar(color="green").encode(
    y=alt.Y("region",axis=alt.Axis(labels=False),title=None),
    x=alt.X("casos_activos",title="promedio de casos activos"),
).properties(
   height=700,width=400
    )

texto_activos=base_activos.mark_text( align='left',
    baseline='middle',
    dx=3 ).encode(
    text=alt.Text('casos_activos',format='.1f'),
)


activosgraph=base_activos + texto_activos

display((confirmadosgraph | fallecidosgraph | activosgraph ).configure_view(stroke="transparent").properties(title="máximos de casos confirmados, máximo de fallecidos y promedio de casos activos por región").configure_title( orient='top', anchor='middle'))

**Comentarios:** Notamos que la región Metropolitana es un outlier en terminos de casos confirmados y fallecidos, pero no para los casos activos.

Notamos que para los casos activos los datos no están tan "concentrados". 

Notamos la similaridad entre el máximo de casos confirmados y el máximo de fallecidos, siendo prácticamente iguales salvo escala, que nos estaría indicando correlación de variables. 

Notamos que el promedio de casos activos no se parece tanto al máximo de fallecidos y máximo de casos confirmados, con varias regiones con proporciones distintas. Esto puede estar asociado a que el promedio de casos activos podría no depender solamente en los casos que hay en el presente, si no que de la manera en que se llego ha esa cifra.

se utilizó este tipo de gráfico por la facilidad de entender comparar lo que pasa entre regiones, así como entre variables, al utilizar area o color para graficar el máximo de casos confirmados o máximo de de fallecidos, se nota que la región metropolitana es más grande que el resto, pero entre las otras regiones puede ser más díficil comparar. Si se separan entre gráficos de distinto tipo, puede ser más díficil comparar entre variables. También se pudieron utilizar scatter plots, pero sería un poco más díficil reconocer entre gráficos la misma región.

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [ ]:
import geopandas as gpd
from pathlib import Path

In [ ]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

In [ ]:
type(regiones)

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [ ]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [ ]:
tasa=confirmados.groupby(["region_id","region","fecha"]).apply(lambda df: (df["tasa"]*df["poblacion"]).sum()/df["poblacion"].sum() ).to_frame().rename(columns={0:"tasa"}).reset_index()

suma_activos=activos[["codigo_region","fecha","casos_activos","poblacion"]].groupby(["codigo_region","fecha"]).agg(sum).reset_index()
display(suma_activos)

suma_fallecidos=fallecidos.replace(region_names).groupby(["region","fecha"]).agg(sum).reset_index()
suma_confirmados=confirmados[["region","fecha","casos_confirmados"]].groupby(["region","fecha"]).agg(sum).reset_index()
display(suma_confirmados)

display(suma_fallecidos)
casos_geo = (
    regiones[["codregion","geometry"]].rename(columns={"codregion":"codigo_region"})
    .merge(suma_activos,on=["codigo_region"])
    .merge(tasa.rename(columns={"region_id":"codigo_region"}),on=["codigo_region","fecha"])
    .merge(suma_fallecidos,on=["region","fecha"])
    .merge(suma_confirmados,on=["region","fecha"])
    
)

casos_geo.head()

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [ ]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [ ]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = casos_geo[lambda df: df["fecha"]==fecha]
    chart = alt.Chart(data).mark_geoshape().encode(
        x="region:N",
        color=alt.Color(col,type="quantitative")
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [ ]:
fecha = "2020-04-13"
col = "tasa" 
covid_chile_chart(fecha, col)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [ ]:
col_widget = widgets.Dropdown(options=['poblacion', 'casos_confirmados', 'tasa', 'casos_activos','fallecidos'],description="Columna",continuous_update=False)

In [ ]:
fecha_widget = widgets.DatePicker(description="Fecha",continuous_update=False)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [ ]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

**Comentarios:** Del gráfico de población notamos la concentración de la población,  a medida que pasa el tiempo para los casos confirmado y fallecidos reflejan esa concentración, los casos activos presentan mayor variabilidad y la tasa, nos permite tener una mejor idea de la gravedad por región destacar la región de Magallanes